In [1]:
from abc import ABCMeta, abstractmethod

In [2]:
class BasePlugin(metaclass=ABCMeta):  
    """
    Атрибут класса supported_formatsи метод run обязаны быть 
    реализованы
    в наследниках этого класса
    """
    @property  
    @abstractmethod  
    def supported_formats(self) -> list:  
        pass  

    @abstractmethod  
    def run(self, input_data: dict):  
        pass

In [3]:
class VideoPlugin(BasePlugin):  
    @property  
    def supported_formats(self) -> list:  
        return ['mp4', 'mkv', 'm8j']  
    
    def run(self):  
        print('Processing video...')


plugin = VideoPlugin()

In [4]:
class MetaClass(type):
 """
 Описание принимаемых параметров:
 mcs–объект метакласса, например <__main__.MetaClass>
 name –строка, имя класса, для которого используется 
данный метакласс, например "User"
 bases –кортеж из классов-родителей, например (SomeMixin, AbstractUser)
 attrs–dict-like объект, хранит в себе значения атрибутов и методов класса
 cls–созданный класс, например <__main__.User>
 extra_kwargs–дополнительные keyword-аргументы переданные в сигнатуру класса
 argsи kwargs–аргументы переданные в конструктор класса 
при создании нового экземпляра
 """
def __new__(mcs, name, bases, attrs, **extra_kwargs):
    return super().__new__(mcs, name, bases, attrs)  

def __init__(cls, name, bases, attrs, **extra_kwargs):  
    super().__init__(cls) 
     
@classmethod  
def __prepare__(mcs, cls, bases, **extra_kwargs):  
    return super().__prepare__(mcs, cls, bases, **kwargs)  

def __call__(cls, *args, **kwargs):  
    return super().__call__(*args, **kwargs)

# Использование метаклассов

Метаклассы позволяют настраивать создание классов в Python. Определив метакласс для класса, вы можете контролировать создание экземпляров класса, его атрибуты и поведение. Метаклассы часто используются для реализации фреймворков, ORM (объектно-реляционных отображений) и других сложных шаблонов программирования.

## Задача 1. Атрибуты класса в верхний регистр
Напишите программу на Python для создания метакласса UpperAttrMeta, который преобразует все имена атрибутов класса в верхний регистр.

In [5]:
class UpperAttrMeta(type):
    def __new__(cls, name, bases, attrs):
        # Создать новый словарь с именами атрибутов в верхнем регистре
        upper_attrs = {k.upper(): v for k, v in attrs.items()}
        
        # Обновить исходные атрибуты до версии в верхнем регистре
        attrs.clear() # очищение
        attrs.update(upper_attrs)
        
        # Вызов суперкласса для фактического создания класса
        return super().__new__(cls, name, bases, attrs)

# Пример использования
class LowerCaseClass(metaclass=UpperAttrMeta):
    normal_attr = "normal"
    another_attr = "another"
    __private_attr = "private"

# Доступ к атрибутам теперь будет осуществляться с помощью печати имен
print(LowerCaseClass.NORMAL_ATTR)  # Вывод: normal
print(LowerCaseClass.ANOTHER_ATTR)  # Вывод: another
# print(LowerCaseClass.__name__)  # Вывод: another, AttributeError
# print(LowerCaseClass.__PRIVATE_ATTR)  # Вывод: private, AttributeError

normal
another


In [6]:
# Определяем метакласс, который преобразует все имена атрибутов в верхний регистр
class UpperAttrMeta(type):
    def __new__(cls, name, bases, dct):
        # Создаем новый словарь для хранения атрибутов в верхнем регистре
        uppercase_attr = {}
        for name, value in dct.items():
            if not name.startswith('__'):
                # Преобразовать название атрибута в верхний регистр
                uppercase_attr[name.upper()] = value
            else:
                # Сохраните специальные методы в том виде, в каком они указаны 
                uppercase_attr[name] = value
        # Создаем новый класс с измененными атрибутами
        return super().__new__(cls, name, bases, uppercase_attr)

# Создаем класс, используя метакласс
class MyClass(metaclass=UpperAttrMeta):
    foo = 'bar'
    baz = 'qux'

# Проверить класс
print(hasattr(MyClass, 'foo'))  # False, потому что имена атрибутов печатаются в верхнем регистре
print(hasattr(MyClass, 'FOO'))  # True, потому что 'foo' преобразуется в 'FOO'
print(MyClass.FOO)  # 'bar', получая доступ к атрибуту верхнего регистра

False
True
bar


## Задача 2. Проверка атрибутов - целых чисел
Напишите программу на Python для создания метакласса «ValidateAttrMeta», который гарантирует, что все атрибуты класса являются целыми числами. Если какой-либо атрибут не является целым числом, вызовите исключение TypeError.

In [7]:
class ValidateAttrMeta(type):
    def __new__(cls, name, bases, attrs):
        # Создаем копию исходных атрибутов
        validated_attrs = attrs.copy()
        
        # Проверяем каждый атрибут
        for attr_name, attr_value in validated_attrs.items():
            # Игнорируем приватные атрибуты
            if not attr_name.startswith('_'):
                # Проверяем, является ли значение целым числом
                if not isinstance(attr_value, int):
                    raise TypeError(f"Aтрибут '{attr_name}' должен быть целым числом.")
        
        # Создаем класс с валидированными атрибутами
        return super().__new__(cls, name, bases, validated_attrs)

# Пример использования
class IntegerOnlyClass(metaclass=ValidateAttrMeta):
    integer_attr = 42
    # float_attr = 3.14
    # string_attr = "hello"
    # list_attr = [1, 2, 3]

# Попытка создания класса с некорректным атрибутом
try:
    class InvalidClass(metaclass=ValidateAttrMeta):
        invalid_attr = "not an integer"
except TypeError as e:
    print(f"Ошибка валидации: {e}")

# Вывод значений атрибутов
print("Значения атрибутов IntegerOnlyClass:")
print(IntegerOnlyClass.integer_attr)
# print(IntegerOnlyClass.float_attr)
# print(IntegerOnlyClass.string_attr)
# print(IntegerOnlyClass.list_attr)

Ошибка валидации: Aтрибут 'invalid_attr' должен быть целым числом.
Значения атрибутов IntegerOnlyClass:
42


In [20]:
# Определить метакласс, который гарантирует, что все атрибуты являются целыми числами
class ValidateAttrMeta(type):
    # Переопределить метод __new__ для метакласса
    def __new__(cls, name, bases, dct):
        # Выполнить итерацию по всем атрибутам класса
        for key, value in dct.items():
            # Проверить, не является ли атрибут специальным методом и не является ли целым числом
            if not key.startswith('__') and not isinstance(value, int):
                # Вызывает ошибку типа, если атрибут не является целым числом
                raise TypeError(f"Attribute {key} is not an integer")
        # Создаем новый класс с проверенными атрибутами
        return super().__new__(cls, name, bases, dct)

# Создаем класс, используя метакласс
class MyIntClass(metaclass=ValidateAttrMeta):
    # Определяем целочисленный атрибут
    foo = 100
    # Определяем другой целочисленный атрибут
    bar = 200
    # Определяем нецелочисленный атрибут (раскомментируйте, чтобы проверить TypeError)
    # baz = 'not an integer'  # При раскомментировании этого параметра должна возникнуть TypeError
    

# Проверьте класс
# Выведите значение атрибута foo
print(MyIntClass.foo)  # 100
# Выведите значение атрибута bar
print(MyIntClass.bar)  # 200
try:
    MyIntClass.baz = 'not an integer'
    print(MyIntClass.baz)
except TypeError as e:
    print(f"Ошибка валидации: {e}")

100
200
not an integer


## Задача 3. Шаблон проектирования «Одиночка»
Напишите программу на Python для создания метакласса SingletonMeta, который гарантирует, что у класса будет только один экземпляр (шаблон проектирования «Одиночка»).

In [6]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(SingletonMeta, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

# Пример использования
class Database(metaclass=SingletonMeta):
    def __init__(self, name):
        self.name = name
        print(f"Initializing database: {name}")

# Создание экземпляров
db1 = Database("MySQL") # создается
db2 = Database("PostgreSQL") # возвращается ссылка на Database("MySQL")

# Проверка наличия только одного экземпляра
print(db1 is db2)  # Выведет: True
print(db1.name)    # Выведет: Initializing database: MySQL
print(db2.name)     # Выведет: Initializing database: MySQL

# Попытка создания нового экземпляра (не должно вызвать инициализации)
db3 = Database("MongoDB") # возвращается ссылка на Database("MySQL")
print(db3.name)      # Выведет: Initializing database: MongoDB

# Проверка идентичности всех экземпляров
db4 = Database("SQLite") # возвращается ссылка на Database("MySQL")
print(db4 is db1)    # Выведет: True

Initializing database: MySQL
True
MySQL
MySQL
MySQL
True


In [6]:
# Определяем метакласс SingletonMeta
class SingletonMeta(type):
    # Словарь для хранения экземпляров классов
    _instances = {}

    # Переопределяем метод __call__ метакласса
    def __call__(cls, *args, **kwargs):
        # Проверяет, не создан ли уже экземпляр класса
        if cls not in cls._instances:
            # Если нет, создайте новый экземпляр и сохраните его в словаре _instances
            cls._instances[cls] = super().__call__(*args, **kwargs)
        # Верните существующий экземпляр, если он уже создан
        return cls._instances[cls]

# Создайте класс SingletonClass, используя SingletonMeta в качестве метакласса
class SingletonClass(metaclass=SingletonMeta):
    pass

# Протестировать класс singleton
# Создайте два экземпляра SingletonClass
instance1 = SingletonClass()
instance2 = SingletonClass()
# Проверьте, ссылаются ли оба экземпляра на один и тот же объект
print(instance1 is instance2)  # True 

True


# Динамическое cоздание класса

Динамическая природа Python позволяет изменять классы и объекты во время выполнения программы. Вы можете добавлять, удалять или изменять атрибуты и методы классов и объектов динамически с помощью встроенных функций, таких как 'setattr', 'getattr' и 'delattr'.

## Задача 4. Динамическое создание класса
Напишите функцию Python «create_class», которая принимает имя класса и словарь атрибутов и методов и возвращает динамически созданный класс с этими атрибутами и методами.

In [21]:
def create_class(class_name, attributes_and_methods):
    # Создаем новый класс с помощью метакласса
    class Meta(type):
        def __new__(meta, name, bases, attrs):
            # Создаем новую переменную для хранения атрибутов
            new_attrs = {}
            
            # Добавляем все атрибуты из входного словаря
            for attr_name, attr_value in attributes_and_methods.items():
                new_attrs[attr_name] = attr_value
            
            # Создаем новый класс с обновленными атрибутами
            return super().__new__(meta, name, (), new_attrs)
    
    # Создаем класс с помощью нашего метакласса
    return Meta(class_name, (), {})

# Пример использования
if __name__ == "__main__":
    # Определение атрибутов и методов
    attributes_and_methods = {
        "x": 10,
        "y": 20,
        "add": lambda self, other: self.x + other,
        "subtract": lambda self, other: self.x - other
    }
    
    # Создание класса
    MyNewClass = create_class("MyNewClass", attributes_and_methods)
    
    # Создание экземпляра и использование методов
    obj = MyNewClass()
    print(obj.x)  # Выведет: 10
    print(obj.y)  # Выведет: 20
    print(obj.add(5))  # Выведет: 15
    print(obj.subtract(3))  # Выведет: 7
    
    # Добавление нового атрибута после создания класса
    MyNewClass.new_attr = "Новый атрибут"
    print(MyNewClass.new_attr)  # Выведет: Новый атрибут

10
20
15
7
Новый атрибут


In [22]:
# Функция для динамического создания класса с заданными атрибутами и методами
def create_class(name, attrs):
    # Используйте функцию type для создания нового класса с заданным именем, наследуемого от object 
    return type(name, (object,), attrs)

# Определите атрибуты и методы для динамического класса
attrs = {
     # Добавить метод 'greet', который возвращает строку
    'greet': lambda self: "Hello, Marsel Sidykov!",
    # Добавить атрибут 'age' со значением 30
    'age': 30
}

# Создать класс динамически, используя определенные атрибуты и методы
MyDynamicClass = create_class('MyDynamicClass', attrs)

# Протестировать динамический класс
# Создаем экземпляр динамически создаваемого класса
instance = MyDynamicClass()
# Вызываем метод 'greet' экземпляра
print(instance.greet())  # Output: "Hello, Sonia Toutatis!"
# Доступ к атрибуту 'age' экземпляра
print(instance.age)  # Output: 25 

Hello, Marsel Sidykov!
30


## Задача 5. Динамическое создание класса
Напишите на Python функцию «add_method», которая принимает класс, имя метода и сам метод и добавляет этот метод в класс.

In [23]:
def add_method(cls, method_name, method):
    setattr(cls, method_name, method)

# Пример использования
class MyClass:
    pass

def greet(self):
    return f"Hello, I'm {self.__class__.__name__}!"

# Добавляем метод greet к MyClass
add_method(MyClass, "greet", greet)

# Создаем экземпляр класса
obj = MyClass()

# Используем новый метод
print(obj.greet())  # Выведет: Hello, I'm MyClass!

Hello, I'm MyClass!


## Задача 6. Динамическое создание класса 
Напишите функцию Python «create_inherited_class», которая принимает базовый класс, имя класса и словарь дополнительных атрибутов и методов и возвращает динамически созданный подкласс.

In [24]:
def create_inherited_class(base_class, class_name, extra_attributes_and_methods):
    # Создаем новый класс, наследуясь от base_class
    NewClass = type(class_name, (base_class,), extra_attributes_and_methods)
    
    return NewClass

# Пример использования
if __name__ == "__main__":
    # Определение базового класса
    class BaseClass:
        def base_method(self):
            return "Это метод базового класса"

    # Определение дополнительных атрибутов и методов
    extra_attributes_and_methods = {
        "new_attribute": "Это новый атрибут",
        "new_method": lambda self: f"Это новый метод в подклассе: {self.__class__.__name__}"
    }

    # Создание подкласса
    SubClass = create_inherited_class(BaseClass, "SubClass", extra_attributes_and_methods)

    # Создание экземпляра подкласса
    sub_instance = SubClass()

    # Использование методов и атрибутов
    print(sub_instance.base_method())  # Вызов метода из базового класса
    print(sub_instance.new_attribute)  # Доступ к новому атрибуту
    print(sub_instance.new_method())   # Вызов нового метода

    # Проверка наследования
    print(isinstance(sub_instance, BaseClass))  # Выведет: True
    print(type(sub_instance) == SubClass)         # Выведет: True

Это метод базового класса
Это новый атрибут
Это новый метод в подклассе: SubClass
True
True


In [25]:
# Функция для создания подкласса с динамическим определением
def create_inherited_class(base_class, name, attrs):
    # Создаем новый класс, наследуемый от base_class, с дополнительными атрибутами и методами
    return type(name, (base_class,), attrs)

# Определяем базовый класс
class BaseClass:
    # Метод, который будет наследоваться подклассами
    def base_method(self):
        return "Base method"

# Определяет дополнительные атрибуты и методы 
additional_attrs = {
    # Добавляет новый метод в подкласс
    'additional_method': lambda self: "Additional method"
}

# Создать подкласс динамически, используя базовый класс и дополнительные атрибуты
DynamicSubclass = create_inherited_class(BaseClass, 'DynamicSubclass', additional_attrs)

# Протестировать динамический подкласс
# Создаем экземпляр динамически создаваемого подкласса
instance = DynamicSubclass()
# Вызываем унаследованный метод из базового класса
print(instance.base_method())       # Вывод: "Base method"
# Вызовите только что добавленный метод в подклассе
print(instance.additional_method())  # Вывод: "Additional method"

Base method
Additional method


# Генерация кода

Функции Python «exec» и «eval» позволяют выполнять динамически сгенерированный код во время выполнения. Эту функцию можно использовать для создания шаблонов кода, анализа предметно-ориентированных языков (DSL) или реализации инструментов для генерации кода

## Задача 7. Генерация кода
Напишите функцию Python «generate_code», которая принимает строку шаблона с заполнителями и словарь значений и возвращает сгенерированный код с заполнителями, заменёнными соответствующими значениями.

In [25]:
def generate_code(template: str, values: dict) -> str:
    """
    Генерирует код, заменяя заполнители в шаблоне соответствующими значениями из словаря.

    :param template: Шаблон строки с заполнителями в формате {placeholder}.
    :param values: Словарь значений для замены заполнителей.
    :return: Сгенерированная строка с заменёнными значениями.
    """
    return template.format(**values)

# Пример использования
template = "Привет, {name}! Сегодня {day}."
values = {
    "name": "Александр",
    "day": "суббота"
}

generated_code = generate_code(template, values)
print(generated_code)


Привет, Александр! Сегодня суббота.


In [ ]:
def generate_code(template_string, values_dict):
    class CodeGenerator(type):
        def __new__(cls, name, bases, dct):
            template = dct.get('__template__', '')
            if template:
                for key, value in values_dict.items():
                    template = template.replace(f'{{{key}}}', str(value))
                dct['__code__'] = compile(template, '<string>', 'exec')
            return super().__new__(cls, name, bases, dct)

    class GeneratedCode(metaclass=CodeGenerator):
        __template__ = template_string

    return GeneratedCode.__code__

# Пример использования:
template_string = """
def generated_function(arg1, arg2):
    print(f"Результат: {arg1} + {arg2}")
"""

values_dict = {
    'arg1': 5,
    'arg2': 3
}

generated_code = generate_code(template_string, values_dict)

# Выполнение сгенерированного кода
exec(generated_code) # выполнение кода через exec

Используйте eval() вместо exec(), так как eval() работает только с выражениями, а не блоками кода

## Задача 8. Генерация кода
Напишите функцию Python «generate_function», которая принимает имя функции и её тело в виде строк и возвращает динамически сгенерированную функцию.

In [26]:
def generate_function(func_name: str, func_body: str):
    """
    Генерирует динамическую функцию с заданным именем и телом.

    :param func_name: Имя создаваемой функции.
    :param func_body: Тело функции в виде строки.
    :return: Динамически созданная функция.
    """
    # Формируем код для создания функции
    code = f"def {func_name}():\n"
    
    # Добавляем тело функции с отступами
    for line in func_body.splitlines():
        code += "    " + line + "\n"
    
    # Выполняем код для создания функции
    exec(code, globals())
    
    # Возвращаем ссылку на созданную функцию
    return globals()[func_name]

# Пример использования
if __name__ == "__main__":
    function_name = "greet"
    function_body = """
print("Привет, мир!")
"""
    
    # Генерация функции
    generated_function = generate_function(function_name, function_body)
    
    # Вызов сгенерированной функции
    generated_function()  # Вывод: Привет, мир!


Привет, мир!


* Использование exec() может быть небезопасным, если входные данные не контролируются, так как это позволяет выполнять произвольный код. Убедитесь, что входные данные безопасны и проверены перед выполнением.
* Динамическое создание функций может быть полезным в определённых случаях, но старайтесь использовать его осторожно и по необходимости.

In [27]:
# Функция для динамической генерации функции
def generate_function(name, body):
    # Выполняет код функции в локальной области
    exec(f"def {name}():\n    {body}")
    # Возвращает сгенерированную функцию из локальной области
    return locals()[name]

# Определяем имя функции и тело
function_name = 'dynamic_function'
function_body = "print('Привет от динамически сгенерированной функции!')"

# Сгенерировать функцию
dynamic_func = generate_function(function_name, function_body)

# Протестировать динамически сгенерированную функцию
dynamic_func()  # Output: "Привет от динамически сгенерированной функции!"

Привет от динамически сгенерированной функции!


## Задача 9. Генерация кода
Напишите функцию Python «generate_inherited_class», которая принимает имя класса, базовый класс и словарь атрибутов и методов и возвращает динамически сгенерированный подкласс.

In [21]:
def generate_inherited_class(class_name: str, base_class: type, attributes_methods: dict) -> type:
    """
    Генерирует динамически подкласс с заданным именем, базовым классом и атрибутами/методами.

    :param class_name: Имя создаваемого класса.
    :param base_class: Базовый класс, от которого будет наследоваться новый класс.
    :param attributes_methods: Словарь с атрибутами и методами класса.
    :return: Динамически созданный подкласс.
    """
    
    # Создание класса с использованием type
    new_class = type(class_name, (base_class,), attributes_methods)
    
    return new_class

# Пример использования
if __name__ == "__main__":
    # Определяем базовый класс
    class BaseClass:
        def base_method(self):
            return "Это метод базового класса."

    # Определяем атрибуты и методы для нового класса
    attributes_methods = {
        "new_attribute": "Это новый атрибут.",
        
        "new_method": lambda self: "Это метод нового класса."
    }

    # Генерация подкласса
    GeneratedClass = generate_inherited_class("DerivedClass", BaseClass, attributes_methods)

    # Создание экземпляра нового класса
    instance = GeneratedClass()

    # Использование атрибута и метода нового класса
    print(instance.new_attribute)  # Вывод: Это новый атрибут.
    print(instance.new_method())   # Вывод: Это метод нового класса.
    print(instance.base_method())   # Вывод: Это метод базового класса.


Это новый атрибут.
Это метод нового класса.
Это метод базового класса.


In [31]:
def generate_inherited_class(base_class_name, base_class, attributes_methods):
    # Create the constructor
    def __init__(self, *args):
        for i, attr in enumerate(attributes_methods['attributes']):
            setattr(self, attr, args[i] if i < len(args) else attributes_methods['attributes'][attr])

    # Create methods dynamically
    methods = {}
    for method_name, method_code in attributes_methods['methods'].items():
        exec(f"def {method_name}(self):\n" + "\n".join(f"    {line}" for line in method_code.splitlines()), methods)

    # Combine the constructor and methods into a dictionary
    methods['__init__'] = __init__

    # Create the new class
    new_class = type(attributes_methods['class_name'], (base_class,), methods)
    
    # Register the new class in globals
    globals()[attributes_methods['class_name']] = new_class

# Example usage:
class BaseClass:
    pass

attributes_methods = {
    'class_name': 'MySubClass',
    'attributes': {'x': 0, 'y': 0},
    'methods': {
        'move': 'self.x += 1\nself.y += 1',
        'get_position': 'return (self.x, self.y)'
    }
}

generate_inherited_class('BaseClass', BaseClass, attributes_methods)

# Create an instance of the new class
instance = globals()[attributes_methods['class_name']](10, 20)
print(instance.get_position())  # Outputs: (10, 20)
instance.move()
print(instance.get_position())  # Outputs: (11, 21)


(10, 20)
(11, 21)


In [32]:
def generate_inherited_class(class_name, base_class, attributes_methods):
    # Формируем код для нового класса
    class_code = f"""
class {class_name}({base_class.__name__}):
    def __init__(self, {', '.join(attributes_methods['attributes'].keys())}):
        super().__init__()
        {self_assignments(attributes_methods['attributes'])}
    
    {methods_definitions(attributes_methods['methods'])}
"""
    
    # Выполняем созданный код
    exec(class_code, globals())
    
    # Возвращаем созданный класс
    return globals()[class_name]

def self_assignments(attributes):
    """Генерирует строки присваивания для атрибутов."""
    return "\n        ".join([f"self.{attr} = {attr}" for attr in attributes])

def methods_definitions(methods):
    """Генерирует определения методов."""
    method_defs = []
    for method_name, method_code in methods.items():
        method_defs.append(f"def {method_name}(self):\n        " + "\n        ".join(method_code.splitlines()))
    return "\n\n    ".join(method_defs)

# Пример использования:
class BaseClass:
    def __init__(self):
        pass

attributes_methods = {
    'attributes': {'x': 0, 'y': 0},
    'methods': {
        'move': 'self.x += 1\nself.y += 1',
        'get_position': 'return (self.x, self.y)'
    }
}

# Генерируем подкласс
MySubClass = generate_inherited_class('MySubClass', BaseClass, attributes_methods)

# Создаем экземпляр нового класса
instance = MySubClass(10, 20)
print(instance.get_position())  # Вывод: (0, 0)
instance.move()
print(instance.get_position())  # Вывод: (1, 1)


(10, 20)
(11, 21)


Таким образом, вы можете динамически создавать подклассы в Python с помощью данной функции.

In [30]:
# Функция для динамического создания подкласса
def generate_inherited_class(name, base_class, attrs):
    # Создайте новый класс, который наследуется от базового класса с дополнительными атрибутами
    return type(name, (base_class,), attrs)

# Определяет базовый класс
class Animal:
    # Метод, который будет унаследован подклассами
    def speak(self):
        return "Animal sound"

# Определяем атрибуты и методы для подкласса
subclass_attrs = {
    # Переопределяем метод speak
    'speak': lambda self: "Сказать 'Гав-Гав'",
    # Добавляем новый атрибут
    'legs': 4
}

# Генерируем подкласс динамически
Dog = generate_inherited_class('Dog', Animal, subclass_attrs)

# Протестируйте динамически созданный подкласс
# Создайте экземпляр Dog
dog = Dog()
# Вызовите метод speak (переопределенный)
print(dog.speak())  # Вывод: "Сказать 'Гав-Гав'"
# Доступ к атрибуту legs
print(dog.legs)     # Вывод: 4

Сказать 'Гав-Гав'
4
